## Extract generic and missing categories from SQL database

In [ ]:
import sqlite3
import json
import os
import pandas as pd
import re
from IPython.display import display  # For Jupyter Notebook display

# SQLite database file path
DB_PATH = "../../data/pipeline2/sql/epd_database.sqlite"
OUTPUT_FILE_PATH = "../../data/pipeline2/sql/filtered_epd_data.csv"

# Allowed Classifications (normalized)
ALLOWED_CATEGORIES = {
    "construction products, infrastructure and buildings",
    "construction products",
    "N/A"  # Handling missing categories
}

# Synonym & Unit Normalization Dictionary
SYNONYM_DICT = {
    "kilogram": "kg",
    "kilograms": "kg",
    "gram": "g",
    "grams": "g",
    "liter": "L",
    "liters": "L",
    "metre": "m",
    "meter": "m",
    "centimetre": "cm",
    "centimeter": "cm",
    "millimetre": "mm",
    "millimeter": "mm"
}

def clean_text(text, lowercase=True):
    """Cleans and normalizes text: removes special characters, extra spaces, and normalizes synonyms & units."""
    if not isinstance(text, str):
        return "N/A"

    text = text.strip().replace("\n", " ")  # Remove newlines and trim spaces
    text = re.sub(r'\s+', ' ', text)  # Remove excessive spaces

    if lowercase:
        text = text.lower()  # Convert to lowercase for consistency
    
    # Normalize punctuation (replace "&" with "and")
    text = text.replace("&", "and")

    # Remove special characters (except spaces, common separators)
    text = re.sub(r'[^a-zA-Z0-9.,\-_/()\s]', '', text)

    # Normalize synonyms and units
    for key, value in SYNONYM_DICT.items():
        text = text.replace(key, value)

    return text

# Function to extract relevant fields from JSON
def extract_epd_data(json_data):
    """Extracts key fields from the EPD JSON document and applies preprocessing & filters."""
    try:
        process_info = json_data.get("processInformation", {})
        dataset_info = process_info.get("dataSetInformation", {})
        classification_info = dataset_info.get("classificationInformation", {}).get("classification", [])
        technology_info = process_info.get("technology", {})

        # Extract base name (product name)
        base_name = clean_text(dataset_info.get("name", {}).get("baseName", [{}])[0].get("value", "N/A"), lowercase=False)

        # Extract classification
        classification = "N/A"
        if classification_info:
            classification = clean_text(classification_info[0]["class"][0].get("value", "N/A"))

        # Debugging: Print all unique classifications to check raw values
        # print(f"DEBUG: Found classification -> {classification}")

        # Extract technology description (manufacturing process)
        technology_description = "N/A"
        if "technologyDescriptionAndIncludedProcesses" in technology_info:
            tech_desc_list = technology_info["technologyDescriptionAndIncludedProcesses"]
            if tech_desc_list:
                technology_description = clean_text(tech_desc_list[0].get("value", "N/A"), lowercase=False)

        # **Filtering Conditions**
        if classification in ALLOWED_CATEGORIES and technology_description != "N/A":
            return {
                "Product Name": base_name,
                "Classification": classification,
                "Technology Description": technology_description
            }
        else:
            return None  # Skip EPDs that don't meet the conditions

    except Exception as e:
        print(f"Error extracting data: {e}")
        return None

# Connect to SQLite and retrieve JSON data
def fetch_epd_data():
    """Fetches and filters EPD data from SQLite."""
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()

        # Fetch all rows from the table
        cursor.execute("SELECT document FROM epd_documents")
        rows = cursor.fetchall()

        extracted_data = []
        unique_classifications = set()

        for row in rows:
            try:
                json_data = json.loads(row[0])  # Parse JSON from SQLite
                epd_info = extract_epd_data(json_data)
                if epd_info:
                    extracted_data.append(epd_info)
                    unique_classifications.add(epd_info["Classification"])  # Store unique classifications for debugging
            except json.JSONDecodeError:
                print("Skipping invalid JSON record.")

        conn.close()

        # Print all unique classification values found
        print("\n### Unique Classifications Found ###")
        for cls in sorted(unique_classifications):
            print(f"- {cls}")

        # Convert extracted data into a DataFrame
        df = pd.DataFrame(extracted_data)

        return df

    except sqlite3.Error as e:
        print(f"SQLite error: {e}")
        return None

# Run the extraction and display results
df_epd = fetch_epd_data()

# Check if the DataFrame is valid
if df_epd is not None and not df_epd.empty:
    display(df_epd)  # Show the data in Jupyter Notebook

    # Ensure the output directory exists
    output_dir = os.path.dirname(OUTPUT_FILE_PATH)
    os.makedirs(output_dir, exist_ok=True)

    # Save DataFrame to CSV
    try:
        df_epd.to_csv(OUTPUT_FILE_PATH, index=False, encoding="utf-8")
        print(f"Filtered EPD data saved to: {OUTPUT_FILE_PATH}")
    except Exception as e:
        print(f"Error saving CSV file: {e}")

else:
    print("No matching EPDs found.")

## Iteration 2

In [ ]:
import sqlite3
import json
import os
import pandas as pd
import re
from IPython.display import display  # For Jupyter Notebook display

# SQLite database file path
DB_PATH = "../../data/pipeline2/sql/epd_database.sqlite"
OUTPUT_FILE_PATH = "../../data/pipeline2/sql/filtered_epd_data03.csv"

# Allowed Classifications (normalized)
ALLOWED_CATEGORIES = {
    "construction products, infrastructure and buildings",
    "construction products",
    ""  # Handling missing categories
}

# Synonym & Unit Normalization Dictionary
SYNONYM_DICT = {
    "kilogram": "kg",
    "kilograms": "kg",
    "gram": "g",
    "grams": "g",
    "liter": "L",
    "liters": "L",
    "metre": "m",
    "meter": "m",
    "centimetre": "cm",
    "centimeter": "cm",
    "millimetre": "mm",
    "millimeter": "mm"
}

def clean_text(text, lowercase=True):
    """Cleans and normalizes text: removes newlines, extra spaces, normalizes punctuation, and standardizes synonyms/units."""
    if not isinstance(text, str):
        return "N/A"

    text = text.strip().replace("\n", " ")          # Remove newlines and trim spaces
    text = re.sub(r'\s+', ' ', text)                  # Remove excessive spaces
    if lowercase:
        text = text.lower()                           # Lowercase text if desired
    # Normalize punctuation (replace "&" with "and")
    text = text.replace("&", "and")
    # Remove special characters (except spaces and common separators)
    text = re.sub(r'[^a-zA-Z0-9.:,\-_/()\s]', '', text)
    # Normalize synonyms and units
    for key, value in SYNONYM_DICT.items():
        text = text.replace(key, value)
    return text

def extract_epd_data(json_data):
    """Extracts key fields from the EPD JSON document, including additional fields."""
    try:
        process_info = json_data.get("processInformation", {})
        dataset_info = process_info.get("dataSetInformation", {})
        classification_info = dataset_info.get("classificationInformation", {}).get("classification", [])
        geography_info = process_info.get("geography", {}).get("locationOfOperationSupplyOrProduction", {})
        technology_info = process_info.get("technology", {})
        modelling_info = json_data.get("modellingAndValidation", {})
        exchanges_info = json_data.get("exchanges", {})

        # Extract key fields
        uuid = dataset_info.get("UUID", "")
        base_name = clean_text(dataset_info.get("name", {}).get("baseName", [{}])[0].get("value", ""), lowercase=False)
        classification = ""
        if classification_info:
            classification = clean_text(classification_info[0]["class"][0].get("value", ""))
        geographic_location = geography_info.get("location", "")
        technology_description = ""
        if "technologyDescriptionAndIncludedProcesses" in technology_info:
            tech_desc_list = technology_info["technologyDescriptionAndIncludedProcesses"]
            if tech_desc_list:
                technology_description = clean_text(tech_desc_list[0].get("value", ""), lowercase=False)
        technological_applicability = ""
        if "technologicalApplicability" in technology_info:
            tech_applicability_list = technology_info["technologicalApplicability"]
            if tech_applicability_list and isinstance(tech_applicability_list, list) and len(tech_applicability_list) > 0:
                technological_applicability = clean_text(tech_applicability_list[0].get("value", ""), lowercase=False)
        lca_method_details = ""
        lci_method_allocation = modelling_info.get("LCIMethodAndAllocation", {})
        ref_to_lca_method_details = lci_method_allocation.get("referenceToLCAMethodDetails", [])
        if ref_to_lca_method_details and isinstance(ref_to_lca_method_details, list) and len(ref_to_lca_method_details) > 0:
            short_desc_list = ref_to_lca_method_details[0].get("shortDescription", [])
            if short_desc_list and isinstance(short_desc_list, list) and len(short_desc_list) > 0:
                lca_method_details = clean_text(short_desc_list[0].get("value", ""), lowercase=False)
        flow_dataset_short_desc = ""
        exchanges = exchanges_info.get("exchange", [])
        if exchanges and isinstance(exchanges, list) and len(exchanges) > 0:
            ref_to_flow_dataset = exchanges[0].get("referenceToFlowDataSet", {})
            short_desc_list = ref_to_flow_dataset.get("shortDescription", [])
            if short_desc_list and isinstance(short_desc_list, list) and len(short_desc_list) > 0:
                flow_dataset_short_desc = clean_text(short_desc_list[0].get("value", ""), lowercase=False)
        flow_property_name = ""
        if exchanges and isinstance(exchanges, list) and len(exchanges) > 0:
            flow_properties = exchanges[0].get("flowProperties", [])
            if flow_properties and isinstance(flow_properties, list) and len(flow_properties) > 0:
                name_list = flow_properties[0].get("name", [])
                if name_list and isinstance(name_list, list) and len(name_list) > 0:
                    flow_property_name = clean_text(name_list[0].get("value", ""), lowercase=False)
        flow_property_mean_value = ""
        if exchanges and isinstance(exchanges, list) and len(exchanges) > 0:
            flow_properties = exchanges[0].get("flowProperties", [])
            if flow_properties and isinstance(flow_properties, list) and len(flow_properties) > 0:
                flow_property_mean_value = flow_properties[0].get("meanValue", "")
        flow_property_reference_unit = ""
        if exchanges and isinstance(exchanges, list) and len(exchanges) > 0:
            flow_properties = exchanges[0].get("flowProperties", [])
            if flow_properties and isinstance(flow_properties, list) and len(flow_properties) > 0:
                flow_property_reference_unit = flow_properties[0].get("referenceUnit", "")
        
        # Filter the data based on allowed classifications.
        if classification in ALLOWED_CATEGORIES:
            return {
                "Product Name": base_name,
                "Technology Description": technology_description,
                "Classification": classification,
                "Technological Applicability": technological_applicability,
                "LCA Method Details": lca_method_details,
                "Flow Dataset Short Description": flow_dataset_short_desc,
                "Flow Property Name": flow_property_name,
                "Flow Property Mean Value": flow_property_mean_value,
                "Flow Property Reference Unit": flow_property_reference_unit,
                "Geographic Location": geographic_location,
                "UUID": uuid,
            }
        else:
            return None

    except Exception as e:
        print(f"Error extracting data: {e}")
        return None

def fetch_epd_data():
    """Fetches and filters EPD data from SQLite."""
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()

        # Fetch all rows from the table
        cursor.execute("SELECT document FROM epd_documents")
        rows = cursor.fetchall()

        extracted_data = []
        unique_classifications = set()

        for row in rows:
            try:
                json_data = json.loads(row[0])
                epd_info = extract_epd_data(json_data)
                if epd_info:
                    extracted_data.append(epd_info)
                    unique_classifications.add(epd_info["Classification"])
            except json.JSONDecodeError:
                print("Skipping invalid JSON record.")

        conn.close()

        print("\n### Unique Classifications Found ###")
        for cls in sorted(unique_classifications):
            print(f"- {cls}")

        df = pd.DataFrame(extracted_data)
        return df

    except sqlite3.Error as e:
        print(f"SQLite error: {e}")
        return None

# Run extraction and display results.
df_epd = fetch_epd_data()

if df_epd is not None and not df_epd.empty:
    display(df_epd)
    output_dir = os.path.dirname(OUTPUT_FILE_PATH)
    os.makedirs(output_dir, exist_ok=True)
    try:
        df_epd.to_csv(OUTPUT_FILE_PATH, index=False, encoding="utf-8")
        print(f"Filtered EPD data saved to: {OUTPUT_FILE_PATH}")
    except Exception as e:
        print(f"Error saving CSV file: {e}")
else:
    print("No matching EPDs found.")


In [ ]:
import pandas as pd
import os

# File path
file_path = "../../data/pipeline2/sql/filtered_epd_data_test.csv"

# Check if file exists
if not os.path.exists(file_path):
    print(f"File not found: {file_path}")
else:
    # Read CSV file
    df_epd = pd.read_csv(file_path)

    # **STEP 1: Clean Text Fields to Remove Extra Spaces & Special Characters**
    def clean_text(text):
        """Ensure uniform text formatting by removing extra spaces and special characters."""
        if isinstance(text, str):
            text = text.strip()  # Remove leading/trailing spaces
            text = text.replace("\n", " ")  # Replace newlines with spaces
            text = " ".join(text.split())  # Remove excessive spaces
            return text
        return "N/A"

    df_epd = df_epd.map(clean_text)  # Apply cleaning to all text fields

    # **STEP 2: Force Left Alignment for ALL Content**
    def left_align_column(column, width):
        """Ensure consistent left alignment by padding each entry to a fixed width."""
        return column.astype(str).str.ljust(width)

    # Define dynamic column widths based on max length in each column
    col_widths = {col: max(df_epd[col].astype(str).apply(len).max(), len(col)) + 2 for col in df_epd.columns}

    # Apply left-alignment formatting to each column
    for col, width in col_widths.items():
        df_epd[col] = left_align_column(df_epd[col], width)

    # **STEP 3: Print Properly Aligned Table**
    print(df_epd.head(20).to_string(index=False))


## Check Category Distribuition

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Define file path
file_path = "../../data/pipeline2/sql/filtered_epd_data02.csv"

# Check if the file exists
if not os.path.exists(file_path):
    print(f"File not found: {file_path}")
else:
    # Read the CSV file
    df_epd = pd.read_csv(file_path)

    # Replace missing values in the Classification column with "Missing"
    df_epd["Classification"] = df_epd["Classification"].fillna("Missing")

    # Get classification counts
    classification_counts = df_epd["Classification"].value_counts()

    # Plot the classification distribution
    plt.figure(figsize=(12, 6))
    ax = classification_counts.plot(kind="bar", edgecolor="black", color="green")

    # Formatting the plot
    plt.xlabel("Category")
    plt.ylabel("Count")
    plt.title("Distribution of Problematic EPD Categories (Including Missing)")
    plt.xticks(rotation=0)
    plt.grid(axis="y", linestyle="--", alpha=0.7)

    # Add count labels inside the bars
    for p in ax.patches:
        # Get bar dimensions
        width = p.get_width()
        height = p.get_height()
        x, y = p.get_xy()
        # Place label in the middle of the bar
        ax.text(x + width/2, y + height/2, int(height), 
                horizontalalignment='center', 
                verticalalignment='center', 
                color='white', fontsize=12)

    # Show the plot
    plt.show()

    # Print category counts in text formats
    print("\n### Category Counts ###")
    for classification, count in classification_counts.items():
        print(f"{classification}: {count}")


## Check Character and Token Counts of `technologyDescriptionAndIncludedProcesses`

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import tiktoken
import numpy as np
import statistics
import os

# File path for the processed EPD data
file_path = "../../data/pipeline2/sql/filtered_epd_data.csv"

# Check if the file exists
if not os.path.exists(file_path):
    print(f"File not found: {file_path}")
else:
    # Read the CSV file
    df_epd = pd.read_csv(file_path)

    # Combine relevant text columns into a single string per row
    df_epd["tech_descr"] = df_epd[["Technology Description"]].astype(str).agg(" ".join, axis=1)

    # Compute character and token counts
    lengths = df_epd["tech_descr"].apply(len).tolist()

    # Initialize tokenizer
    enc = tiktoken.get_encoding("o200k_base")
    token_counts = [len(enc.encode(text)) for text in df_epd["tech_descr"]]

    # Basic statistics
    mean_length = statistics.mean(lengths) if lengths else 0
    mean_tokens = statistics.mean(token_counts) if token_counts else 0

    median_length = np.median(lengths) if lengths else 0
    median_tokens = np.median(token_counts) if token_counts else 0

    std_length = np.std(lengths, ddof=1) if len(lengths) > 1 else 0
    std_tokens = np.std(token_counts, ddof=1) if len(token_counts) > 1 else 0

    percentiles_length = np.percentile(lengths, [25, 50, 75, 90, 95]) if lengths else [0] * 5
    percentiles_tokens = np.percentile(token_counts, [25, 50, 75, 90, 95]) if token_counts else [0] * 5

    iqr_length = percentiles_length[2] - percentiles_length[0]
    iqr_tokens = percentiles_tokens[2] - percentiles_tokens[0]

    # Print statistical measures
    print(f"Mean character count: {mean_length:.2f}")
    print(f"Median character count: {median_length:.2f}")
    print(f"Standard deviation (chars): {std_length:.2f}")
    print(f"25th, 50th, 75th, 90th, 95th percentiles (chars): {percentiles_length}")
    print(f"IQR (chars): {iqr_length:.2f}\n")

    print(f"Mean token count: {mean_tokens:.2f}")
    print(f"Median token count: {median_tokens:.2f}")
    print(f"Standard deviation (tokens): {std_tokens:.2f}")
    print(f"25th, 50th, 75th, 90th, 95th percentiles (tokens): {percentiles_tokens}")
    print(f"IQR (tokens): {iqr_tokens:.2f}\n")

    # Plot histograms for visual inspection
    plt.figure(figsize=(10, 5))
    plt.hist(lengths, bins=20, edgecolor="black")
    plt.title("Character Count Distribution")
    plt.xlabel("Character Count")
    plt.ylabel("Frequency")
    plt.show()

    plt.figure(figsize=(10, 5))
    plt.hist(token_counts, bins=20, edgecolor="black")
    plt.title("Token Count Distribution")
    plt.xlabel("Token Count")
    plt.ylabel("Frequency")
    plt.show()

    # Print histogram data in numeric format
    def print_histogram_data(data, bins=20, title="Histogram"):
        counts, bin_edges = np.histogram(data, bins=bins)
        print(title)
        for i in range(len(counts)):
            lower = bin_edges[i]
            upper = bin_edges[i + 1]
            print(f"{lower:5.1f} - {upper:5.1f}: {counts[i]}")

    print_histogram_data(lengths, bins=20, title="Character Count Histogram Data")
    print_histogram_data(token_counts, bins=20, title="Token Count Histogram Data")


## Summarize `technologyDescriptionAndIncludedProcesses`

In [ ]:
import json
import os
import pandas as pd
from langchain_ollama import ChatOllama  # Use your Ollama integration
from langchain_core.prompts import ChatPromptTemplate

# Define a JSON schema for the summarization response.
json_schema = {
    "title": "SummarizationResponse",
    "description": "Response containing a summarized version of the technology description.",
    "type": "object",
    "properties": {
        "summary": {
            "type": "string",
            "description": "A concise summary of the technology description emphasizing key raw materials and its use in construction."
        }
    },
    "required": ["summary"]
}

# Create a prompt template that only uses the Technology Description.
prompt_template = ChatPromptTemplate.from_template(
    """
You are an expert in summarizing technical manufacturing processes for building material products.
Summarize the following technology description into a concise paragraph that emphasizes:
- The key raw materials (e.g., PMMA, titanium dioxide, aluminum hydroxide, etc.)
Avoid excessive process details; focus on the material composition and intended use.

Technology Description:
{text}

Return your answer in JSON format following this schema:
{json_schema}
"""
)

# Set your Ollama model name (adjust as needed for your environment)
llm_model = "deepseek-r1:8b"  # e.g., "deepseek-r1:8b"
model = ChatOllama(model=llm_model)

def summarize_technology_description(text):
    """Uses Ollama to summarize a long Technology Description."""
    final_prompt = prompt_template.format_prompt(
        text=text, json_schema=json.dumps(json_schema)
    ).to_string()
    
    structured_llm = model.with_structured_output(
        json_schema, method="json_schema", include_raw=True
    )
    raw_response = structured_llm.invoke(final_prompt)
    structured_response = raw_response.get("parsed", None)
    if structured_response and "summary" in structured_response:
        return structured_response["summary"]
    return None

# Path to the CSV file.
csv_path = "../../data/pipeline2/sql/filtered_epd_data.csv"

number_entries = 100

# Read the CSV file and select only the first 100 entries.
df = pd.read_csv(csv_path).head(number_entries)

# Container to hold all responses.
summaries = []

# Process each row.
for idx, row in df.iterrows():
    tech_description = row.get("Technology Description", "")
    if tech_description and isinstance(tech_description, str):
        summary = summarize_technology_description(tech_description)
        # Create a dictionary for this row, including a row index for reference.
        summaries.append({
            "row_index": idx,
            "Product Name": row.get("Product Name", ""),
            "Classification": row.get("Classification", ""),
            "Technology Description Summary": summary
        })
        print(f"Summary for row {idx}:\n{summary}\n{'-'*50}\n")
    else:
        print(f"Row {idx} has no Technology Description.")

# Ensure the output directory exists.
output_dir = os.path.join("..", "..", "data", "pipeline2", "json")
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, f"{number_entries}_technology_summaries.json")

# Write the JSON responses to file.
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(summaries, f, indent=2)

print(f"All summaries saved to {output_file}")


### Iteration 2

In [ ]:
import json
import os
import pandas as pd
from langchain_ollama import ChatOllama  # Use your Ollama integration
from langchain_core.prompts import ChatPromptTemplate

# Define a JSON schema for the summarization response.
json_schema = {
    "title": "SummarizationResponse",
    "description": "Response containing a summarized version of the technology description.",
    "type": "object",
    "properties": {
        "summary": {
            "type": "string",
            "description": "A concise summary of the technology description emphasizing key raw materials and its use in construction."
        }
    },
    "required": ["summary"]
}

# Create a prompt template that only uses the Technology Description.
prompt_template = ChatPromptTemplate.from_template(
    """
You are an expert in summarizing technical documents of building material products to facilitate categorization.
Summarize the following technology description into a concise paragraph emphasizing key raw materials and its use in construction.
If no materials are mentioned, avoid excessive process details.

Technology Description:
{text}

Return your answer in JSON format following this schema:
{json_schema}
"""
)

# Set your Ollama model name (adjust as needed for your environment)
llm_model = "deepseek-r1:8b"  # e.g., "deepseek-r1:8b"
model = ChatOllama(model=llm_model)

def summarize_technology_description(text):
    """Uses Ollama to summarize a long Technology Description."""
    final_prompt = prompt_template.format_prompt(
        text=text, json_schema=json.dumps(json_schema)
    ).to_string()
    
    structured_llm = model.with_structured_output(
        json_schema, method="json_schema", include_raw=True
    )
    raw_response = structured_llm.invoke(final_prompt)
    structured_response = raw_response.get("parsed", None)
    if structured_response and "summary" in structured_response:
        return structured_response["summary"]
    return None

# Path to the CSV file.
csv_path = "../../data/pipeline2/sql/filtered_epd_data.csv"

number_entries = 100

# Read the CSV file and select only the first 100 entries.
df = pd.read_csv(csv_path).head(number_entries)

# Container to hold all responses.
summaries = []

# Process each row.
for idx, row in df.iterrows():
    tech_description = row.get("Technology Description", "")
    if tech_description and isinstance(tech_description, str):
        summary = summarize_technology_description(tech_description)
        # Create a dictionary for this row, including a row index for reference.
        summaries.append({
            "row_index": idx,
            "Product Name": row.get("Product Name", ""),
            "Classification": row.get("Classification", ""),
            "Technology Description Summary": summary
        })
        print(f"Summary for row {idx}:\n{summary}\n{'-'*50}\n")
    else:
        print(f"Row {idx} has no Technology Description.")

# Ensure the output directory exists.
output_dir = os.path.join("..", "..", "data", "pipeline2", "json")
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, f"{number_entries}_tech_sum_one_paragraph.json")

# Write the JSON responses to file.
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(summaries, f, indent=2)

print(f"All summaries saved to {output_file}")


## Extract Composition Materials from `technologyDescriptionAndIncludedProcesses`

In [ ]:
import json
import os
import pandas as pd
from langchain_ollama import ChatOllama  # Use your Ollama integration
from langchain_core.prompts import ChatPromptTemplate

# Define a JSON schema for the composition extraction response.
json_schema = {
    "title": "CompositionExtractionResponse",
    "description": "Response containing a list of key composition materials extracted from the technology description.",
    "type": "object",
    "properties": {
        "composition": {
            "type": "string",
            "description": "A list of key raw materials mentioned in the technology description, e.g., PMMA, titanium dioxide, aluminum hydroxide."
        }
    },
    "required": ["composition"]
}

# Create a prompt template that focuses on extracting composition materials from the Technology Description.
prompt_template = ChatPromptTemplate.from_template(
    """
You are an expert in technical manufacturing processes for building material products.
Extract and list the key composition materials from the following technology description. 
Focus exclusively on the raw materials (e.g., PMMA, titanium dioxide, aluminum hydroxide, etc.) used in the product.
Avoid including unnecessary process details; only list the materials.

Technology Description:
{text}

Return your answer in JSON format following this schema:
{json_schema}
"""
)

# Set your Ollama model name (adjust as needed for your environment)
llm_model = "deepseek-r1:8b"  # for example, "deepseek-r1:8b"
model = ChatOllama(model=llm_model)

def extract_composition_materials(text):
    """Uses Ollama to extract the composition materials from a Technology Description."""
    final_prompt = prompt_template.format_prompt(
        text=text, json_schema=json.dumps(json_schema)
    ).to_string()
    
    structured_llm = model.with_structured_output(
        json_schema, method="json_schema", include_raw=True
    )
    raw_response = structured_llm.invoke(final_prompt)
    structured_response = raw_response.get("parsed", None)
    if structured_response and "composition" in structured_response:
        return structured_response["composition"]
    return None

# Path to the CSV file.
csv_path = "../../data/pipeline2/sql/filtered_epd_data.csv"

number_entries = 100

# Read the CSV file and select only the first 100 entries.
df = pd.read_csv(csv_path).head(number_entries)

# Container to hold all responses.
composition_extractions = []

# Process each row.
for idx, row in df.iterrows():
    tech_description = row.get("Technology Description", "")
    if tech_description and isinstance(tech_description, str):
        composition = extract_composition_materials(tech_description)
        # Create a dictionary for this row, including a row index for reference.
        composition_extractions.append({
            "row_index": idx,
            "Product Name": row.get("Product Name", ""),
            "Classification": row.get("Classification", ""),
            "Extracted Composition": composition
        })
        print(f"Extracted composition for row {idx}:\n{composition}\n{'-'*50}\n")
    else:
        print(f"Row {idx} has no Technology Description.\n{'-'*50}\n")

# Ensure the output directory exists.
output_dir = os.path.join("..", "..", "data", "pipeline2", "json")
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, f"{number_entries}_technology_compositions.json")

# Write the JSON responses to file.
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(composition_extractions, f, indent=2)

print(f"All composition extractions saved to {output_file}")

## Do Both Summarization and Composition Material Extraction from `technologyDescriptionAndIncludedProcesses`

In [ ]:
import json
import os
import pandas as pd
from langchain_ollama import ChatOllama  # Use your Ollama integration
from langchain_core.prompts import ChatPromptTemplate

# Define a JSON schema for the combined summarization and composition extraction response.
json_schema = {
    "title": "SummarizationAndCompositionResponse",
    "description": "Response containing a summarized version of the technology description and a list of key composition materials.",
    "type": "object",
    "properties": {
        "summary": {
            "type": "string",
            "description": "A concise summary of the technology description emphasizing key raw materials and its use in construction."
        },
        "composition": {
            "type": "string",
            "description": "A list of key raw materials mentioned in the technology description, e.g., PMMA, titanium dioxide, aluminum hydroxide."
        }
    },
    "required": ["summary", "composition"]
}

# Create a prompt template that instructs the model to provide both outputs.
prompt_template = ChatPromptTemplate.from_template(
    """
You are an expert in technical manufacturing processes for building material products.
Given the following technology description, perform two tasks:
1. Summarize the description into a concise paragraph that emphasizes the key raw materials.
2. Extract and list the key composition materials mentioned (e.g., PMMA, titanium dioxide, aluminum hydroxide).

Avoid excessive process details; focus on material composition and intended use.

Technology Description:
{text}

Return your answer in JSON format following this schema:
{json_schema}
"""
)

# Set your Ollama model name (this is dynamic so you can test different models)
llm_model = "deepseek-r1:8b"  # For example, "deepseek-r1:8b", "falcon3:7b-instruct-q4_K_M",
model = ChatOllama(model=llm_model)

def process_technology_description(text):
    """Uses Ollama to get both a summary and the extracted composition materials from a Technology Description."""
    final_prompt = prompt_template.format_prompt(
        text=text, json_schema=json.dumps(json_schema)
    ).to_string()

    structured_llm = model.with_structured_output(
        json_schema, method="json_schema", include_raw=True
    )
    raw_response = structured_llm.invoke(final_prompt)
    structured_response = raw_response.get("parsed", None)
    if structured_response and "summary" in structured_response and "composition" in structured_response:
        return structured_response
    return {"summary": None, "composition": None}

# Path to the CSV file.
csv_path = os.path.join("..", "..", "data", "pipeline2", "sql", "filtered_epd_data.csv")

number_entries = 100

# Read the CSV file and select only the first 100 entries.
df = pd.read_csv(csv_path).head(number_entries)

# Container to hold all responses.
results = []

# Process each row.
for idx, row in df.iterrows():
    tech_description = row.get("Technology Description", "")
    if tech_description and isinstance(tech_description, str):
        response = process_technology_description(tech_description)
        results.append({
            "row_index": idx,
            "Product Name": row.get("Product Name", ""),
            "Classification": row.get("Classification", ""),
            "Technology Description Summary": response.get("summary"),
            "Extracted Composition": response.get("composition")
        })
        print(f"Summary and material composition for row {idx}:\n{response.get("summary")}\n{response.get("composition")}\n{'-'*50}\n")
    else:
        print(f"Row {idx} has no Technology Description.")

# Ensure the output directory exists.
output_dir = os.path.join("..", "..", "data", "pipeline2", "json")
os.makedirs(output_dir, exist_ok=True)

# Include the model name in the output file name.
model_name = llm_model.split(":")[0]
output_file = os.path.join(output_dir, f"{number_entries}_technology_summary_composition_{model_name}.json")

# Write the JSON responses to file.
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2)

print(f"All responses saved to {output_file}")


## Summarize starting with Product Name

In [ ]:
import json
import os
import pandas as pd
from langchain_ollama import ChatOllama  # Use your Ollama integration
from langchain_core.prompts import ChatPromptTemplate

# Define a JSON schema for the summarization response (only a summary field).
json_schema = {
    "title": "TechnologySummaryResponse",
    "description": "Response containing a concise technology summary that begins with the product name.",
    "type": "object",
    "properties": {
        "summary": {
            "type": "string",
            "description": "A concise summary of the technology description that starts with the product name."
        }
    },
    "required": ["summary"]
}

# Create a prompt template that uses both the Product Name and Technology Description.
prompt_template = ChatPromptTemplate.from_template(
    """
You are an expert in technical manufacturing processes for building material products.
Using the information provided, produce a concise technology summary that begins with the product name.
The summary should start with "The product {product_name}" and then describe the manufacturing process and key raw materials.

Technology Description: {text}

Return your answer in JSON format following this schema:
{json_schema}
"""
)

# Set your Ollama model name (this can be changed dynamically)
llm_model = "deepseek-r1:8b"  # e.g., "deepseek-r1:8b" or another model name
model = ChatOllama(model=llm_model)

def summarize_technology_description(product_name, text):
    """Uses Ollama to generate a summary for the technology description that starts with the product name."""
    final_prompt = prompt_template.format_prompt(
        product_name=product_name,
        text=text,
        json_schema=json.dumps(json_schema)
    ).to_string()
    
    structured_llm = model.with_structured_output(
        json_schema, method="json_schema", include_raw=True
    )
    raw_response = structured_llm.invoke(final_prompt)
    structured_response = raw_response.get("parsed", None)
    if structured_response and "summary" in structured_response:
        return structured_response["summary"]
    return None

# Path to the CSV file.
csv_path = os.path.join("..", "..", "data", "pipeline2", "sql", "filtered_epd_data.csv")
number_entries = 100

# Read the CSV file and select only the first 100 entries.
df = pd.read_csv(csv_path).head(number_entries)

# Container to hold all responses.
results = []

# Process each row.
for idx, row in df.iterrows():
    product_name = row.get("Product Name", "")
    tech_description = row.get("Technology Description", "")
    if product_name and tech_description and isinstance(tech_description, str):
        summary = summarize_technology_description(product_name, tech_description)
        results.append({
            "row_index": idx,
            "Product Name": product_name,
            "Classification": row.get("Classification", ""),
            "Technology Description Summary": summary
        })
        print(f"Processed row {idx}\n{product_name}\n\n{summary}\n{'-'*50}\n")
    else:
        print(f"Row {idx} missing product name or technology description.")

# Ensure the output directory exists.
output_dir = os.path.join("..", "..", "data", "pipeline2", "json")
os.makedirs(output_dir, exist_ok=True)

# Include the model name in the output file name.
model_name = llm_model.split(":")[0]
output_file = os.path.join(output_dir, f"{number_entries}_tech_sum_prodname_{model_name}.json")

# Write the JSON responses to file.
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2)

print(f"All summaries saved to {output_file}")

### Iteration 2

In [ ]:
import json
import os
import pandas as pd
from langchain_ollama import ChatOllama  # Use your Ollama integration
from langchain_core.prompts import ChatPromptTemplate

# Define a JSON schema for the summarization response (only a summary field).
json_schema = {
    "title": "TechnologySummaryResponse",
    "description": "Response containing a concise technology summary that begins with the product name.",
    "type": "object",
    "properties": {
        "summary": {
            "type": "string",
            "description": "A concise summary of the technology description that starts with the product name."
        }
    },
    "required": ["summary"]
}

# Create a prompt template that uses both the Product Name and Technology Description.
prompt_template = ChatPromptTemplate.from_template(
    """
You are an expert in technical manufacturing processes for building material products.
Using the information provided, produce a one sentence summary that begins with the product name.
The summary should start with "The product {product_name}" and then describe concisely the product.

Technology Description: {text}

Return your answer in JSON format following this schema:
{json_schema}
"""
)

# Set your Ollama model name (this can be changed dynamically)
llm_model = "deepseek-r1:8b"  # e.g., "deepseek-r1:8b" or another model name
model = ChatOllama(model=llm_model)

def summarize_technology_description(product_name, text):
    """Uses Ollama to generate a summary for the technology description that starts with the product name."""
    final_prompt = prompt_template.format_prompt(
        product_name=product_name,
        text=text,
        json_schema=json.dumps(json_schema)
    ).to_string()
    
    structured_llm = model.with_structured_output(
        json_schema, method="json_schema", include_raw=True
    )
    raw_response = structured_llm.invoke(final_prompt)
    structured_response = raw_response.get("parsed", None)
    if structured_response and "summary" in structured_response:
        return structured_response["summary"]
    return None

# Path to the CSV file.
csv_path = os.path.join("..", "..", "data", "pipeline2", "sql", "filtered_epd_data.csv")
number_entries = 100

# Read the CSV file and select only the first 100 entries.
df = pd.read_csv(csv_path).head(number_entries)

# Container to hold all responses.
results = []

# Process each row.
for idx, row in df.iterrows():
    product_name = row.get("Product Name", "")
    tech_description = row.get("Technology Description", "")
    if product_name and tech_description and isinstance(tech_description, str):
        summary = summarize_technology_description(product_name, tech_description)
        results.append({
            "row_index": idx,
            "Product Name": product_name,
            "Classification": row.get("Classification", ""),
            "Technology Description Summary": summary
        })
        print(f"Processed row {idx}\n{product_name}\n\n{summary}\n{'-'*50}\n")
    else:
        print(f"Row {idx} missing product name or technology description.")

# Ensure the output directory exists.
output_dir = os.path.join("..", "..", "data", "pipeline2", "json")
os.makedirs(output_dir, exist_ok=True)

# Include the model name in the output file name.
model_name = llm_model.split(":")[0]
output_file = os.path.join(output_dir, f"{number_entries}_tech_sum_prodname_{model_name}.json")

# Write the JSON responses to file.
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2)

print(f"All summaries saved to {output_file}")

## Summarize Technology Descriptions in one Sentence

In [ ]:
import json
import os
import pandas as pd
from langchain_ollama import ChatOllama  # Use your Ollama integration
from langchain_core.prompts import ChatPromptTemplate

# Define a JSON schema for the summarization response.
json_schema = {
    "title": "SummarizationResponse",
    "description": "Response containing a summarized version of the technology description.",
    "type": "object",
    "properties": {
        "summary": {
            "type": "string",
            "description": "A concise summary of the technology description emphasizing key raw materials and its use in construction."
        }
    },
    "required": ["summary"]
}

# Create a prompt template that only uses the Technology Description.
prompt_template = ChatPromptTemplate.from_template(
    """
You are an expert in summarizing technical documents of building material products.
Summarize the following technology description into a concise sentence that facilitates the categorization of the building product.

Technology Description:
{text}

Return your answer in JSON format following this schema:
{json_schema}
"""
)

# Set your Ollama model name (adjust as needed for your environment)
llm_model = "deepseek-r1:8b"  # e.g., "deepseek-r1:8b"
model = ChatOllama(model=llm_model)

def summarize_technology_description(text):
    """Uses Ollama to summarize a long Technology Description."""
    final_prompt = prompt_template.format_prompt(
        text=text, json_schema=json.dumps(json_schema)
    ).to_string()
    
    structured_llm = model.with_structured_output(
        json_schema, method="json_schema", include_raw=True
    )
    raw_response = structured_llm.invoke(final_prompt)
    structured_response = raw_response.get("parsed", None)
    if structured_response and "summary" in structured_response:
        return structured_response["summary"]
    return None

# Path to the CSV file.
csv_path = "../../data/pipeline2/sql/filtered_epd_data.csv"

number_entries = 100

# Read the CSV file and select only the first 100 entries.
df = pd.read_csv(csv_path).head(number_entries)

# Container to hold all responses.
summaries = []

# Process each row.
for idx, row in df.iterrows():
    tech_description = row.get("Technology Description", "")
    if tech_description and isinstance(tech_description, str):
        summary = summarize_technology_description(tech_description)
        # Create a dictionary for this row, including a row index for reference.
        summaries.append({
            "row_index": idx,
            "Product Name": row.get("Product Name", ""),
            "Classification": row.get("Classification", ""),
            "Technology Description Summary": summary
        })
        print(f"Summary for row {idx}:\n{summary}\n{'-'*50}\n")
    else:
        print(f"Row {idx} has no Technology Description.")

# Ensure the output directory exists.
output_dir = os.path.join("..", "..", "data", "pipeline2", "json")
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, f"{number_entries}_tech_sum_one_sentence.json")

# Write the JSON responses to file.
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(summaries, f, indent=2)

print(f"All summaries saved to {output_file}")


### Iteration 2

In [ ]:
import json
import os
import pandas as pd
from langchain_ollama import ChatOllama  # Use your Ollama integration
from langchain_core.prompts import ChatPromptTemplate

# Define a JSON schema for the summarization response.
json_schema = {
    "title": "SummarizationResponse",
    "description": "Response containing a summarized version of the technology description.",
    "type": "object",
    "properties": {
        "summary": {
            "type": "string",
            "description": "A concise summary of the technology description emphasizing key raw materials and its use in construction."
        }
    },
    "required": ["summary"]
}

# Create a prompt template that only uses the Technology Description.
prompt_template = ChatPromptTemplate.from_template(
    """
You are an expert in summarizing technical documents of building material products to facilitate categorization.
Summarize the following technology description into a concise sentence emphasizing key raw materials and its use in construction.

Technology Description:
{text}

Return your answer in JSON format following this schema:
{json_schema}
"""
)

# Set your Ollama model name (adjust as needed for your environment)
llm_model = "deepseek-r1:8b"  # e.g., "deepseek-r1:8b"
model = ChatOllama(model=llm_model)

def summarize_technology_description(text):
    """Uses Ollama to summarize a long Technology Description."""
    final_prompt = prompt_template.format_prompt(
        text=text, json_schema=json.dumps(json_schema)
    ).to_string()
    
    structured_llm = model.with_structured_output(
        json_schema, method="json_schema", include_raw=True
    )
    raw_response = structured_llm.invoke(final_prompt)
    structured_response = raw_response.get("parsed", None)
    if structured_response and "summary" in structured_response:
        return structured_response["summary"]
    return None

# Path to the CSV file.
csv_path = "../../data/pipeline2/sql/filtered_epd_data.csv"

number_entries = 100

# Read the CSV file and select only the first 100 entries.
df = pd.read_csv(csv_path).head(number_entries)

# Container to hold all responses.
summaries = []

# Process each row.
for idx, row in df.iterrows():
    tech_description = row.get("Technology Description", "")
    if tech_description and isinstance(tech_description, str):
        summary = summarize_technology_description(tech_description)
        # Create a dictionary for this row, including a row index for reference.
        summaries.append({
            "row_index": idx,
            "Product Name": row.get("Product Name", ""),
            "Classification": row.get("Classification", ""),
            "Technology Description Summary": summary
        })
        print(f"Summary for row {idx}:\n{summary}\n{'-'*50}\n")
    else:
        print(f"Row {idx} has no Technology Description.")

# Ensure the output directory exists.
output_dir = os.path.join("..", "..", "data", "pipeline2", "json")
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, f"{number_entries}_tech_sum_one_sentence02.json")

# Write the JSON responses to file.
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(summaries, f, indent=2)

print(f"All summaries saved to {output_file}")


## Print Character and Token Count

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import tiktoken
import numpy as np
import statistics
import os

# File path for the JSON summaries
json_file_path = os.path.join("..", "..", "data", "pipeline2", "json", "100_technology_summaries.json")

# Check if the file exists
if not os.path.exists(json_file_path):
    print(f"File not found: {json_file_path}")
else:
    # Load JSON data
    with open(json_file_path, "r", encoding="utf-8") as f:
        summaries = json.load(f)
    
    # Convert JSON list to a DataFrame
    df = pd.DataFrame(summaries)
    
    # We assume that the summarized text is under the key "Technology Description Summary"
    # If your key is different (e.g., "summary"), adjust accordingly.
    if "Technology Description Summary" not in df.columns:
        print("Key 'Technology Description Summary' not found in JSON data.")
    else:
        # Compute the character count of each summary.
        df["char_count"] = df["Technology Description Summary"].astype(str).apply(len)
        
        # Initialize the tokenizer.
        enc = tiktoken.get_encoding("o200k_base")
        df["token_count"] = df["Technology Description Summary"].astype(str).apply(lambda text: len(enc.encode(text)))
        
        # Collect counts into lists.
        lengths = df["char_count"].tolist()
        token_counts = df["token_count"].tolist()
        
        # Calculate basic statistics.
        mean_length = statistics.mean(lengths) if lengths else 0
        mean_tokens = statistics.mean(token_counts) if token_counts else 0
        median_length = np.median(lengths) if lengths else 0
        median_tokens = np.median(token_counts) if token_counts else 0
        std_length = np.std(lengths, ddof=1) if len(lengths) > 1 else 0
        std_tokens = np.std(token_counts, ddof=1) if len(token_counts) > 1 else 0
        percentiles_length = np.percentile(lengths, [25, 50, 75, 90, 95]) if lengths else [0] * 5
        percentiles_tokens = np.percentile(token_counts, [25, 50, 75, 90, 95]) if token_counts else [0] * 5
        iqr_length = percentiles_length[2] - percentiles_length[0]
        iqr_tokens = percentiles_tokens[2] - percentiles_tokens[0]
        
        # Print statistical measures.
        print(f"Mean character count: {mean_length:.2f}")
        print(f"Median character count: {median_length:.2f}")
        print(f"Standard deviation (chars): {std_length:.2f}")
        print(f"25th, 50th, 75th, 90th, 95th percentiles (chars): {percentiles_length}")
        print(f"IQR (chars): {iqr_length:.2f}\n")
        
        print(f"Mean token count: {mean_tokens:.2f}")
        print(f"Median token count: {median_tokens:.2f}")
        print(f"Standard deviation (tokens): {std_tokens:.2f}")
        print(f"25th, 50th, 75th, 90th, 95th percentiles (tokens): {percentiles_tokens}")
        print(f"IQR (tokens): {iqr_tokens:.2f}\n")
        
        # Plot histogram for character counts.
        plt.figure(figsize=(10, 5))
        plt.hist(lengths, bins=20, edgecolor="black")
        plt.title("Character Count Distribution of Summaries")
        plt.xlabel("Character Count")
        plt.ylabel("Frequency")
        plt.show()
        
        # Plot histogram for token counts.
        plt.figure(figsize=(10, 5))
        plt.hist(token_counts, bins=20, edgecolor="black")
        plt.title("Token Count Distribution of Summaries")
        plt.xlabel("Token Count")
        plt.ylabel("Frequency")
        plt.show()
        
        # Function to print histogram data numerically.
        def print_histogram_data(data, bins=20, title="Histogram"):
            counts, bin_edges = np.histogram(data, bins=bins)
            print(title)
            for i in range(len(counts)):
                lower = bin_edges[i]
                upper = bin_edges[i + 1]
                print(f"{lower:5.1f} - {upper:5.1f}: {counts[i]}")
        
        print_histogram_data(lengths, bins=20, title="Character Count Histogram Data")
        print_histogram_data(token_counts, bins=20, title="Token Count Histogram Data")


# Categories

In [ ]:
import requests
import xml.etree.ElementTree as ET
from urllib.parse import quote

# Base URL for the API endpoints
BASE_URL = "https://www.oekobaudat.de/OEKOBAU.DAT/resource/processes/categories/"

def get_categories(path):
    """
    Fetches categories from the API.
    - If path is empty, gets top-level categories.
    - Otherwise, retrieves subcategories for the given path.
    """
    if path:
        # Encode each segment in the path to handle spaces and special characters
        encoded_path = "/".join(quote(segment) for segment in path.split("/"))
        url = f"{BASE_URL}{encoded_path}/subcategories/?catSystem=oekobau.dat&lang=en&sort=id"
    else:
        url = f"{BASE_URL}?catSystem=oekobau.dat&lang=en&sort=id"
    
    # Debug print: Show the URL being requested
    print(f"Fetching URL: {url}")
    
    response = requests.get(url)
    response.raise_for_status()  # Stop if the request fails
    return ET.fromstring(response.content)

def build_tree(path, xml_parent, level=0):
    """
    Recursively builds the XML tree:
    - 'path' is used to construct the API URL.
    - 'xml_parent' is the XML element to which child categories will be added.
    - 'level' indicates the depth for debugging purposes.
    """
    indent = "  " * level
    print(f"{indent}Processing path: '{path or 'Top-level'}'")
    
    try:
        root = get_categories(path)
    except Exception as e:
        print(f"{indent}Error retrieving categories for path '{path}': {e}")
        return

    # Define namespace for parsing API XML
    ns = {"sapi": "http://www.ilcd-network.org/ILCD/ServiceAPI"}
    # Find all category elements regardless of their level
    for cat in root.findall(".//sapi:category", ns):
        cat_name = cat.text.strip() if cat.text else "Unknown"
        cat_id = cat.get("classId", "N/A")
        print(f"{indent}  Found category: {cat_name} (ID: {cat_id})")
        
        # Create a new XML element in our output with attributes id and name
        new_elem = ET.SubElement(xml_parent, "category", id=cat_id, name=cat_name)
        # Construct new path for recursion
        new_path = cat_name if not path else f"{path}/{cat_name}"
        # Recursively build tree for subcategories
        build_tree(new_path, new_elem, level + 1)

def main():
    # Register namespaces for the output file
    ET.register_namespace("", "http://lca.jrc.it/ILCD/Categories")
    ET.register_namespace("common", "http://lca.jrc.it/ILCD/Common")
    ET.register_namespace("xsi", "http://www.w3.org/2001/XMLSchema-instance")
    
    # Create the root element <CategorySystem> with attribute name="OEKOBAU.DAT"
    root_elem = ET.Element("CategorySystem", {"name": "OEKOBAU.DAT"})
    # Create the <categories> container with dataType="Process"
    categories_elem = ET.SubElement(root_elem, "categories", {"dataType": "Process"})
    
    print("Starting category tree traversal...")
    # Start building the tree from the top-level (empty path)
    build_tree("", categories_elem)
    
    # Write the constructed XML tree to a file with an XML declaration
    tree = ET.ElementTree(root_elem)
    output_file = "categories.xml"
    tree.write(output_file, encoding="UTF-8", xml_declaration=True)
    print(f"Finished. Categories saved to {output_file}")

if __name__ == "__main__":
    main()


In [ ]:
import xml.dom.minidom

# Read the existing XML file
with open("categories.xml", "rb") as f:
    xml_data = f.read()

# Parse the XML using minidom
dom = xml.dom.minidom.parseString(xml_data)

# Pretty print the XML with desired indentation
pretty_xml = dom.toprettyxml(indent="    ", encoding="UTF-8")

# Write the prettified XML to a new file
with open("categories_pretty.xml", "wb") as f:
    f.write(pretty_xml)

print("Prettified XML saved to categories_pretty.xml")


In [ ]:
import requests
import xml.etree.ElementTree as ET
from urllib.parse import quote
import xml.dom.minidom

# Optimize to not go past level 2
# Problems getting all German categories because of umlauts in the URL 

# Base URL for the API endpoints
BASE_URL = "https://www.oekobaudat.de/OEKOBAU.DAT/resource/processes/categories/"

def get_categories(path):
    """
    Fetches categories from the API.
    - If path is empty, gets top-level categories.
    - Otherwise, retrieves subcategories for the given path.
    """
    if path:
        encoded_path = "/".join(quote(segment) for segment in path.split("/"))
        url = f"{BASE_URL}{encoded_path}/subcategories/?catSystem=oekobau.dat&lang=en&sort=id"
    else:
        url = f"{BASE_URL}?catSystem=oekobau.dat&lang=en&sort=id"
    
    print(f"Fetching URL: {url}")
    response = requests.get(url)
    response.raise_for_status()  # Raise error for bad responses
    return ET.fromstring(response.content)

def build_tree(path, xml_parent, level=0):
    """
    Recursively builds the XML tree:
    - 'path' is used to construct the API URL.
    - 'xml_parent' is the XML element to which child categories will be added.
    - 'level' indicates the recursion depth (for debugging).
    """
    indent = "  " * level
    print(f"{indent}Processing path: '{path or 'Top-level'}'")
    
    try:
        root = get_categories(path)
    except Exception as e:
        print(f"{indent}Error retrieving categories for path '{path}': {e}")
        return

    ns = {"sapi": "http://www.ilcd-network.org/ILCD/ServiceAPI"}
    for cat in root.findall(".//sapi:category", ns):
        cat_name = cat.text.strip() if cat.text else "Unknown"
        cat_id = cat.get("classId", "N/A")
        print(f"{indent}  Found category: {cat_name} (ID: {cat_id})")
        
        new_elem = ET.SubElement(xml_parent, "category", id=cat_id, name=cat_name)
        new_path = cat_name if not path else f"{path}/{cat_name}"
        build_tree(new_path, new_elem, level + 1)

def main():
    # Register namespaces for the output file
    ET.register_namespace("", "http://lca.jrc.it/ILCD/Categories")
    ET.register_namespace("common", "http://lca.jrc.it/ILCD/Common")
    ET.register_namespace("xsi", "http://www.w3.org/2001/XMLSchema-instance")
    
    # Create the root element <CategorySystem> with attribute name="OEKOBAU.DAT"
    root_elem = ET.Element("CategorySystem", {"name": "OEKOBAU.DAT"})
    # Create the <categories> container with dataType="Process"
    categories_elem = ET.SubElement(root_elem, "categories", {"dataType": "Process"})
    
    print("Starting category tree traversal...")
    build_tree("", categories_elem)
    
    # Convert the ElementTree to a string and pretty print it using minidom
    xml_str = ET.tostring(root_elem, encoding="utf-8")
    parsed = xml.dom.minidom.parseString(xml_str)
    pretty_xml_str = parsed.toprettyxml(indent="    ", encoding="UTF-8")
    
    # Write the pretty XML to file
    with open("categories_en_pretty.xml", "wb") as f:
        f.write(pretty_xml_str)
    
    print("Finished. Categories saved to categories.xml")

if __name__ == "__main__":
    main()
